In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_absolute_error, classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [27]:
# Загрузка данных и формирование целевой переменной
df = pd.read_csv('D:/my_ML/diploma_polytech/data/raw/vehicle_ins_data_1.csv', sep = ";",index_col= False)
# Анализ данных перед обработкой
print("=== АНАЛИЗ ДАННЫХ ===")
print(f"Исходный размер: {df.shape}")
print(f"Распределение N_claims_year:\n{df['N_claims_year'].value_counts().sort_index()}")

# Преобразуем даты
df['Date_start_contract'] = pd.to_datetime(df['Date_start_contract'], errors='coerce')

# Формируем целевую переменную
df['claim_event'] = (df['N_claims_year'] > 1).astype(int)
print(f"Распределение целевой переменной:")
print(df['claim_event'].value_counts(normalize=True))

# ИСКЛЮЧАЕМ ВСЕ ВРЕМЕННЫЕ ПРИЗНАКИ И ПОТЕНЦИАЛЬНЫЕ УТЕЧКИ
leakage_features = [
    'Cost_claims_year', 'N_claims_history', 'R_Claims_history' # временной признак - может утекать информация
]
drop_cols = leakage_features + [
    'ID_policy', 'Date_start_contract', 'End_date', 'Date_birth', 'Renew_date', 'N_claims_year'
]

# Отбираем только безопасные признаки
base_features = ['Distribution_channel', 'Seniority', 'Policies_in_force', 'Max_policies', 
                'Max_products', 'Lapse', 'Payment', 'Premium', 'Type_risk', 'Area', 
                'Second_driver', 'Power', 'Cylinder_capacity', 'Value_vehicle', 
                'N_doors', 'Type_fuel', 'Length', 'Weight']

features = [f for f in base_features if f not in drop_cols]
print(f"Используется {len(features)} признаков: {features}")

# Подготовка данных
df_clean = df[features + ['claim_event', 'Date_start_contract']].dropna()  # временно оставляем Start_date для разделения

# Временное разделение
train_df = df_clean[df_clean['Date_start_contract'] < '2017-07-01'].copy()
test_df = df_clean[df_clean['Date_start_contract'] >= '2017-07-01'].copy()

print(f"\nРазмеры выборок:")
print(f"Train: {train_df.shape}, Test: {test_df.shape}")
print(f"Доля положительных классов в train: {train_df['claim_event'].mean():.3f}")
print(f"Доля положительных классов в test: {test_df['claim_event'].mean():.3f}")

# Удаляем Start_date после разделения
train_df = train_df.drop(columns=['Date_start_contract'])
test_df = test_df.drop(columns=['Date_start_contract'])



=== АНАЛИЗ ДАННЫХ ===
Исходный размер: (105555, 30)
Распределение N_claims_year:
N_claims_year
0     85909
1      9539
2      4961
3      2435
4      1190
5       609
6       318
7       227
8       136
9        82
10       61
11       24
12       18
13       18
14       10
15        8
16        2
17        3
18        2
19        1
21        1
25        1
Name: count, dtype: int64
Распределение целевой переменной:
claim_event
0    0.904249
1    0.095751
Name: proportion, dtype: float64
Используется 18 признаков: ['Distribution_channel', 'Seniority', 'Policies_in_force', 'Max_policies', 'Max_products', 'Lapse', 'Payment', 'Premium', 'Type_risk', 'Area', 'Second_driver', 'Power', 'Cylinder_capacity', 'Value_vehicle', 'N_doors', 'Type_fuel', 'Length', 'Weight']

Размеры выборок:
Train: (33237, 20), Test: (7267, 20)
Доля положительных классов в train: 0.107
Доля положительных классов в test: 0.087


C:\Users\andre\AppData\Local\Temp\ipykernel_6884\3020517239.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/my_ML/diploma_polytech/data/raw/vehicle_ins_data_1.csv', sep = ";",index_col= False)


In [28]:
# Кодируем категориальные переменные
categorical = ['Distribution_channel', 'Type_risk', 'Type_fuel']
categorical = [c for c in categorical if c in train_df.columns]

train_df = pd.get_dummies(train_df, columns=categorical, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical, drop_first=True)

# Выравниваем столбцы (на случай разных категорий в train/test)
common_cols = train_df.columns.intersection(test_df.columns).drop('claim_event')
train_df = train_df[common_cols.tolist() + ['claim_event']]
test_df = test_df[common_cols.tolist() + ['claim_event']]

print(f"После кодирования: train {train_df.shape}, test {test_df.shape}")

# Разделяем X и y
X_train, y_train = train_df.drop('claim_event', axis=1), train_df['claim_event']
X_test, y_test = test_df.drop('claim_event', axis=1), test_df['claim_event']

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




После кодирования: train (33237, 23), test (7267, 23)


In [29]:
# Перевод в torch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1,1)

# DataLoader
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)


In [30]:
# Упрощенная модель с регуляризацией
n_features = X_train_tensor.shape[1]
model = nn.Sequential(
    nn.Linear(n_features, 32),
    nn.ReLU(),
    nn.Dropout(0.4),  # Увеличиваем dropout
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(16, 1)
)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 регуляризация

In [31]:
# Обучение с мониторингом переобучения
print("\n=== ОБУЧЕНИЕ ===")
for epoch in range(15):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # Валидация после каждой эпохи
    model.eval()
    with torch.no_grad():
        train_logits = model(X_train_tensor)
        train_proba = torch.sigmoid(train_logits).numpy().flatten()
        train_auc = roc_auc_score(y_train, train_proba)
        
        test_logits = model(X_test_tensor)
        test_proba = torch.sigmoid(test_logits).numpy().flatten()
        test_auc = roc_auc_score(y_test, test_proba)
    
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_loader):.4f}, train_auc={train_auc:.4f}, test_auc={test_auc:.4f}")


=== ОБУЧЕНИЕ ===
Epoch 1: loss=0.4634, train_auc=0.6298, test_auc=0.5985
Epoch 2: loss=0.3455, train_auc=0.6935, test_auc=0.6514
Epoch 3: loss=0.3386, train_auc=0.7064, test_auc=0.6580
Epoch 4: loss=0.3310, train_auc=0.7119, test_auc=0.6604
Epoch 5: loss=0.3301, train_auc=0.7152, test_auc=0.6601
Epoch 6: loss=0.3249, train_auc=0.7183, test_auc=0.6620
Epoch 7: loss=0.3242, train_auc=0.7200, test_auc=0.6613
Epoch 8: loss=0.3230, train_auc=0.7218, test_auc=0.6609
Epoch 9: loss=0.3205, train_auc=0.7233, test_auc=0.6632
Epoch 10: loss=0.3210, train_auc=0.7241, test_auc=0.6630
Epoch 11: loss=0.3206, train_auc=0.7253, test_auc=0.6643
Epoch 12: loss=0.3200, train_auc=0.7260, test_auc=0.6641
Epoch 13: loss=0.3205, train_auc=0.7267, test_auc=0.6648
Epoch 14: loss=0.3172, train_auc=0.7272, test_auc=0.6641
Epoch 15: loss=0.3177, train_auc=0.7278, test_auc=0.6656


In [34]:
# Финальная оценка
model.eval()
with torch.no_grad():
    test_logits = model(X_test_tensor)
    test_proba = torch.sigmoid(test_logits).numpy().flatten()
    predictions = (test_proba > 0.5).astype(int)

auc = roc_auc_score(y_test, test_proba)
mae = mean_absolute_error(y_test, test_proba)

print("\n=== ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ ===")
print(f"ROC-AUC: {auc:.4f}")
print(f"MAE: {mae:.4f}")

# Детальная диагностика
print("\n=== ДИАГНОСТИКА ===")
print("Classification Report:")
print(classification_report(y_test, predictions))

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

# Проверка на тренировочных данных
with torch.no_grad():
    train_proba = torch.sigmoid(model(X_train_tensor)).numpy().flatten()
train_auc = roc_auc_score(y_train, train_proba)
print(f"\nROC-AUC на тренировочных данных: {train_auc:.4f}")
print(f"Разница train-test AUC: {train_auc - auc:.4f}")

# Анализ распределения предсказаний
print(f"\nРаспределение предсказанных вероятностей:")
print(f"Min: {test_proba.min():.3f}, Max: {test_proba.max():.3f}")
print(f"Mean: {test_proba.mean():.3f}, Std: {test_proba.std():.3f}")


=== ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ ===
ROC-AUC: 0.6656
MAE: 0.2010

=== ДИАГНОСТИКА ===
Classification Report:
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      6637
           1       0.00      0.00      0.00       630

    accuracy                           0.91      7267
   macro avg       0.46      0.50      0.48      7267
weighted avg       0.83      0.91      0.87      7267

Confusion Matrix:
[[6637    0]
 [ 630    0]]

ROC-AUC на тренировочных данных: 0.7278
Разница train-test AUC: 0.0621

Распределение предсказанных вероятностей:
Min: 0.000, Max: 0.466
Mean: 0.148, Std: 0.078


d:\My_ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\My_ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\My_ML\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
